# Multi-agent Customer Support Automation

Six key elements which helps to make Agents perform better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


In [4]:
from langchain.chat_models import ChatOpenAI

# Creating Agents

## Role Playing, Focus and Cooperation

In [51]:
financial_support_agent = Agent(
    role="Global Tax Compliance Specialist",
    goal="Provide expert guidance on international tax regulations and compliance",
    backstory=(
        "An experienced tax consultant specializing in international finance. "
        "You help businesses understand complex tax implications across different "
        "jurisdictions and provide clear, actionable guidance for compliance."
    ),
    verbose=True,
    allow_delegation=False,
    llm=ChatOpenAI(
        openai_api_key=OPENAI_API_KEY,
        model_name='gpt-4o'
    )
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task.

In [52]:
financial_qa_agent = Agent(
    role="Tax Advisory Quality Assurance Specialist",
    goal="Ensure accuracy and compliance of tax guidance",
    backstory=(
        "A former Big Four tax auditor with extensive experience in "
        "reviewing international tax compliance strategies. You ensure "
        "all advice meets regulatory standards and best practices."
    ),
    verbose=True,
    llm=ChatOpenAI(
        openai_api_key=OPENAI_API_KEY,
        model_name='gpt-4o'
    )
)


* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: financial_qa_agent can delegate work back to the financial_support_agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

In [53]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [54]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://www.investopedia.com/terms/t/taxation.asp"
)
knowledge_tool = ScrapeWebsiteTool(
    website_url="https://www.investopedia.com/financial-term-dictionary-4769738"
)


##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

# Creating Tasks

You are passing the Tool on the Task Level.

In [55]:
tax_advisory_task = Task(
    description=(
        "Customer {customer} through {person} has requested tax guidance:\n"
        "{inquiry}\n\n"
        "Provide comprehensive tax advisory based on available resources."
    ),
    expected_output=(
        "Detailed tax guidance including:\n"
        "- Analysis of tax implications\n"
        "- Jurisdiction-specific considerations\n"
        "- Compliance requirements\n"
        "- Implementation recommendations"
    ),
    tools=[docs_scrape_tool, knowledge_tool],
    agent=financial_support_agent,
)

In [56]:
quality_review_task = Task(
    description=(
        "Review the provided tax guidance for accuracy, completeness, "
        "and compliance with international tax regulations."
    ),
    expected_output=(
        "Comprehensive review including:\n"
        "- Validation of tax advice\n"
        "- Compliance verification\n"
        "- Risk assessment\n"
        "- Final recommendations"
    ),
    # tools=[docs_scrape_tool, knowledge_tool],
    agent=financial_qa_agent,
)

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [76]:
from langchain_openai import OpenAIEmbeddings

In [77]:
# Configure embedding model for memory
embedding_model = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model_name='text-embedding-ada-002'
)


In [82]:
tax_advisory_crew = Crew(
    agents=[financial_support_agent, financial_qa_agent],
    tasks=[tax_advisory_task, quality_review_task],
    verbose=2,
    # memory = True,
    embedding=embedding_model

)


### Running the Crew

### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [83]:
inputs = {
        "customer": "Global Tech Solutions",
        "person": "Sarah Chen",
        "inquiry": "We're expanding our business to Europe and Asia. "
                  "Need guidance on VAT implications and local tax requirements "
                  "for our software services. How should we structure our "
                  "international tax compliance?"
    }

result = tax_advisory_crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Global Tax Compliance Specialist
 [INFO]: == Starting Task: Customer Global Tech Solutions through Sarah Chen has requested tax guidance:
We're expanding our business to Europe and Asia. Need guidance on VAT implications and local tax requirements for our software services. How should we structure our international tax compliance?

Provide comprehensive tax advisory based on available resources.


> Entering new CrewAgentExecutor chain...
I need to gather information on taxation and financial terms to provide a comprehensive tax advisory for Global Tech Solutions. This will help in understanding VAT implications, local tax requirements, and international tax compliance for their software services in Europe and Asia.

Action: Read website content
Action Input: {"url": "https://www.investopedia.com/terms/t/taxation.asp"}
 

Taxation Defined, With Justifications and Types of Taxes
Skip to content
 Investing
 Stocks
 Cryptocurrency
 Bonds
 ETFs
 Options and Deri

In [59]:
from IPython.display import Markdown
Markdown(result)

The guidance for Global Tech Solutions provides a comprehensive overview of the VAT implications and compliance requirements for expanding software services to Europe and Asia. For Europe, it correctly emphasizes the importance of understanding the EU VAT directive and the 'place of supply' rules, which determine the VAT obligations based on where the customer is established, rather than where the supplier is located. This means that Global Tech Solutions must register for VAT in each EU member state where they supply software services, and the VAT rates can indeed range from 17% to 27%, depending on the country.

In Asia, the guidance accurately reflects the diversity in VAT/GST rates and compliance requirements across different jurisdictions. For example, in China, the VAT for software services can range from 6% to 13%, depending on the specific type of software service provided and the applicable tax incentives. In India, the Goods and Services Tax (GST) is uniformly applied at 18% for software services, but businesses must be aware of the registration threshold, which is currently INR 20 lakhs (approximately USD 27,000) for service providers.

The guidance also correctly outlines the key compliance processes, which include VAT/GST registration in each relevant jurisdiction, issuing compliant invoices, maintaining proper documentation, and filing regular VAT/GST returns. It wisely recommends engaging local advisors to navigate the complex local tax landscapes and implementing robust systems to ensure ongoing compliance.

Overall, the guidance provides a solid foundation for understanding the VAT implications and compliance requirements for Global Tech Solutions as they expand their software services to Europe and Asia. It is crucial for the company to stay informed about any changes in tax legislation in these regions and to continuously assess their compliance strategies to avoid potential penalties or legal issues.

---

## Overview

CrewAI is built to orchestrate autonomous AI agents so that they work together as a “crew” to solve complex tasks. Two core features that empower these agents are:

1. **Tools** – These are modular functionalities (or “skills”) that extend what an agent can do. Tools let agents interact with external data sources, perform computations, read or write files, scrape websites, and even interface with APIs or other software systems.
2. **Memory** – CrewAI endows agents with a multi‐layered memory system. This lets them remember recent interactions, store long‐term learnings, and keep track of key entities or contextual details. Such memory is vital for maintaining continuity in conversations or across task executions.

---

## Tools in CrewAI

### Purpose and Design

In CrewAI, tools serve as the “action modules” that agents use to accomplish specific tasks. They are designed with the following characteristics:
- **Utility:** Tools are built to handle a wide range of operations—from web scraping to database searches and code execution.
- **Integration:** Tools are integrated seamlessly into an agent’s workflow. An agent’s role can be extended by assigning one or more tools that match the kind of task the agent needs to perform.
- **Customizability:** Developers can use pre-built tools or create custom ones. For example, you might subclass a base tool (using CrewAI’s `BaseTool`) or decorate a function with the provided `@tool` decorator to quickly turn it into an actionable module.
- **Error Handling and Caching:** Each tool is built with robust error handling to ensure that if something goes wrong (for instance, a network call fails), the agent can gracefully recover. Caching mechanisms may also be used to optimize repeated operations.

### Types of Tools Available

CrewAI’s documentation lists a broad array of tools that cover many use cases. Here are some representative categories:

- **Web and Data Loaders:**  
  - *Browserbase Web Loader:* Loads web content for analysis.  
  - *EXA Search Web Loader:* Facilitates web searching with retrieval-augmented generation (RAG).

- **Document and File Handlers:**  
  - *Directory Read & Directory RAG Search:* Enable agents to access and search through file directories.  
  - *DOCX, PDF, TXT RAG Search:* Specialized tools to extract and analyze information from different document formats.  
  - *File Read and File Write:* Basic tools to read from or write to files.

- **Code and Data Analysis Tools:**  
  - *Code Docs RAG Search & Code Interpreter:* Let agents search through code documentation or even execute code snippets to generate results.  
  - *CSV RAG Search, JSON RAG Search, MySQL RAG Search, NL2SQL Tool, PG RAG Search:* These help agents work with structured data stored in various formats or databases.

- **Scraping and Search Tools:**  
  - *Firecrawl (Crawl, Scrape, and Search) Tools, Selenium Scraper, Spider Scraper:* Designed for web scraping and content extraction tasks.  
  - *Github Search, Google Serper Search:* Allow agents to query repositories or search engines respectively.

- **Media Tools:**  
  - *DALL-E Tool & Vision Tool:* Interface with generative image models or perform visual analysis.  
  - *YouTube Channel and Video RAG Search:* Enable agents to find and analyze video content.

- **Additional Integrations:**  
  - CrewAI also supports integration with third-party tool ecosystems (for example, LangChain and LlamaIndex tools) to further expand agent capabilities.

By equipping agents with these tools, CrewAI allows them to interact with external APIs, process varied data types, and even execute code. This “toolbox” is essential for enabling agents to perform actions beyond mere text generation.

---

## Memory in CrewAI

### Why Memory Matters

Memory in CrewAI is not a single monolithic feature—it is a layered system that supports different types of information retention. This helps agents:
- **Maintain Context:** By remembering what has just happened, an agent can generate coherent responses in ongoing tasks or conversations.
- **Learn Over Time:** Long-term learnings help agents improve decision-making and adjust strategies based on past successes or errors.
- **Understand Entities:** Keeping track of specific entities (like customer names, products, or topics) helps the agent relate new inputs to previous interactions.

### Types of Memory

CrewAI implements several memory components:

1. **Short-Term Memory:**  
   - **Function:** Temporarily stores recent interactions, intermediate results, or context that is only needed during a single execution session.  
   - **Usage:** For example, during a multi-step conversation or task, agents can refer back to recent dialogue or actions.
   - **Mechanism:** Often implemented using retrieval-augmented generation (RAG) techniques so that the most immediately relevant information is quickly accessible.

2. **Long-Term Memory:**  
   - **Function:** Persists valuable insights and learnings from past sessions or tasks over multiple executions.  
   - **Usage:** This allows agents to accumulate experience and refine their performance over time. For instance, remembering user preferences or frequently encountered problems.
   - **Storage:** Typically uses a persistent database (such as SQLite) that stores task results or summaries that survive between sessions.

3. **Entity Memory:**  
   - **Function:** Specifically captures and organizes information about entities encountered during tasks—such as names, locations, or other key concepts.  
   - **Usage:** Helps in building relationship maps and deepening the agent’s understanding of its environment.
   - **Mechanism:** Also uses RAG approaches to store and later retrieve structured data about entities.

4. **Contextual Memory:**  
   - **Function:** Combines short-term, long-term, and entity memories into a unified context.  
   - **Usage:** Ensures that as a conversation or task sequence progresses, the agent maintains a coherent and contextually appropriate narrative.
   - **Benefit:** This integrated view is critical when tasks require multi-step reasoning, as it lets the agent “remember” earlier parts of the interaction.

### Memory Operations

Beyond just storing data, CrewAI’s memory system supports several operations:
- **Reading:** Agents extract needed information from memory to inform current decisions.
- **Writing:** As new interactions occur, agents record information that might be useful in the future.
- **Reflection:** Agents can summarize or analyze what has been stored, allowing them to “learn” from past actions and adjust strategies accordingly.

### Customization and Integration

- **Custom Configurations:** Memory is configurable; for example, you can enable it simply by setting `memory=True` when creating a Crew. You can also customize the embedding models used for memory (with providers like OpenAI, Ollama, or others).
- **User Memory:** There are also approaches (like integrating with Mem0) for maintaining personalized memory across sessions, which can tailor interactions for individual users.

---

## Conclusion

CrewAI’s power lies in its two-pronged approach:
- **Tools** extend the capabilities of each agent, letting them interact with the world by performing actions such as web searches, file operations, or executing code.
- **Memory** gives agents the ability to maintain context, learn from past experiences, and understand the entities involved in their tasks.

Together, these features make CrewAI a flexible and powerful framework for building multi-agent systems that are both contextually aware and action-capable.

For further details, you can explore the [CrewAI Tools documentation](https://docs.crewai.com/core-concepts/tools) and the [Memory documentation](https://docs.crewai.com/core-concepts/memory) on the CrewAI website.  


## **1. Tools in CrewAI** 🔧
Tools enable agents to interact with external systems/data. CrewAI supports multiple tool types:

### **A. Built-in Tools**
| Tool Type                | Purpose                                                                 | Example                                                                 |
|--------------------------|-------------------------------------------------------------------------|-------------------------------------------------------------------------|
| **Search Tools**          | Web search capabilities                                                 | `SerperDevTool` (Google Search API)                                     |
| **Scraping Tools**        | Extract content from websites                                           | `ScrapeWebsiteTool` (Full page scraping)                                |
| **Website Search**        | Search specific websites                                                | `WebsiteSearchTool` (Domain-specific search)                            |
| **File Operations**       | Read/write local files                                                  | `FileReadTool`, `FileWriteTool`                                         |
| **API Interaction**       | Connect to REST APIs                                                    | `APIOperationTool` (GET/POST requests)                                  |
| **Code Execution**        | Run code snippets                                                       | `PythonREPLTool` (Execute Python code)                                  |

### **B. Custom Tools**
Create tools using decorators or subclassing:
```python
from crewai_tools import BaseTool

# Method 1: Decorator
@tool("Password Generator")
def generate_password(length: int = 12) -> str:
    """Generates secure random password"""
    import secrets, string
    return ''.join(secrets.choice(string.ascii_letters + string.digits + "!@#$%") for _ in range(length))

# Method 2: Subclass
class PDFAnalyzer(BaseTool):
    name = "PDF Analyzer"
    description = "Extracts text and metadata from PDF files"
    
    def _run(self, file_path: str) -> str:
        from PyPDF2 import PdfReader
        reader = PdfReader(file_path)
        return f"Text: {reader.pages[0].extract_text()}\nPages: {len(reader.pages)}"
```

### **C. Tool Configuration**
```python
from crewai_tools import SerperDevTool

search_tool = SerperDevTool(
    api_key="serper_api_key",  # Required for paid tools
    params={"gl": "us", "hl": "en"}  # Optional parameters
)
```

---

## **2. Memory Systems 🧠**
CrewAI uses layered memory for context preservation:

### **A. Memory Types**
| Type                | Scope           | Storage Mechanism                     | Access Pattern              |
|---------------------|-----------------|----------------------------------------|-----------------------------|
| **Short-Term**      | Current Task    | In-memory conversation history        | Immediate context window    |
| **Long-Term**       | Cross-execution | Vector store (e.g., FAISS, Pinecone)  | Semantic similarity search   |
| **Episodic**        | Agent-specific  | Task-specific experiences             | Direct recall               |
| **Procedural**      | Crew-wide       | Successful workflows                  | Pattern matching            |

### **B. Memory Configuration**
```python
from crewai import Crew
from langchain_openai import OpenAIEmbeddings

# 1. Configure Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# 2. Initialize Crew with Memory
crew = Crew(
    agents=[...],
    tasks=[...],
    memory=True,  # Enable both memory types
    embedding=embeddings,  # Required for vector storage
    memory_options={
        'max_context_length': 4096,  # Token limit
        'retention_cycles': 3  # Keep last 3 conversations
    }
)
```

### **C. Memory Access Patterns**
1. **Automatic Context Injection**
```python
agent = Agent(
    role="Researcher",
    memory_boost=True  # Auto-inject relevant memories
)
```

2. **Manual Memory Query**
```python
# Get related memories for current task
context = crew.retrieve_memories(
    query="Latest cryptocurrency regulations",
    k=5  # Top 5 relevant memories
)
```

3. **Memory Pruning**
```python
crew.prune_memories(
    cutoff_date="2024-01-01",  # Remove older entries
    importance_threshold=0.7
)
```

---

## **3. Advanced Patterns** 🚀

### **A. Tool + Memory Integration**
```python
@tool("Research Assistant")
def web_researcher(query: str) -> str:
    """Searches web and stores findings in memory"""
    results = search_tool.run(query)
    crew.store_memory(f"Web research for {query}", results)
    return results
```

### **B. Custom Vector Stores**
```python
from langchain_community.vectorstores import Pinecone

vector_store = Pinecone.from_existing_index(
    index_name="crewai-memories",
    embedding=embeddings
)

crew = Crew(
    ...,
    vector_store=vector_store
)
```

### **C. Memory Security**
```python
# Enable encrypted memory storage
crew = Crew(
    ...,
    memory_security={
        'encryption_key': os.getenv("MEMORY_KEY"),
        'access_control': {
            'agent_1': ['read'],
            'agent_2': ['read', 'write']
        }
    }
)
```

---

## **Key Implementation Notes** 📝
1. **Tool Dependencies**: Many tools require API keys (Serper, OpenAI, etc.)
2. **Memory Overhead**: Vector storage increases memory usage by ~15-20%
3. **Performance Tradeoffs**:
   - `memory=True`: +30-50ms per agent interaction
   - Complex tools add 100-500ms latency
4. **Error Handling**: Implement tool fallbacks:
```python
tool = SerperDevTool()
tool.configure(fallback_strategy="cached_results")
```

For complete details, refer to the [Official CrewAI Documentation](https://docs.crewai.com/introduction).

---

### **1. Tools in CrewAI**
Tools allow agents to interact with external systems. Here's the exact implementation per documentation:

#### **A. Creating Tools**
```python
from crewai_tools import tool

@tool("Email Validator")
def email_validator(email: str) -> bool:
    """Validates email addresses using regex"""
    import re
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return bool(re.match(pattern, email))
```

#### **B. Using Built-in Tools**
```python
from crewai_tools import (
    ScrapeWebsiteTool,
    SerperDevTool,
    PDFSearchTool
)

# Web Scraping Tool
scrape_tool = ScrapeWebsiteTool()

# Google Search Tool (requires Serper API key)
search_tool = SerperDevTool(api_key="your_serper_key")

# PDF Search Tool
pdf_tool = PDFSearchTool(pdf_path="data.pdf")
```

#### **C. Assigning Tools to Agents**
```python
from crewai import Agent

researcher = Agent(
    role="Senior Researcher",
    goal="Make best research about AI",
    tools=[search_tool, scrape_tool],  # Tool assignment
    verbose=True
)
```

---

### **2. Memory System**
CrewAI's memory helps agents retain context between executions.

#### **A. Enabling Memory**
```python
from crewai import Crew

crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    memory=True,  # Enable memory
    verbose=2
)
```

#### **B. Memory Configuration**
```python
from langchain.embeddings import OpenAIEmbeddings

crew = Crew(
    # ... other params ...
    memory=True,
    embedding_model=OpenAIEmbeddings(model="text-embedding-3-small")
)
```

#### **C. Memory Types (Documented)**
1. **Short-term Memory**: Last 5 interactions
2. **Long-term Memory**: Vector store using specified embeddings

---

### **3. Core Concepts from Documentation**

#### **A. Agent Components**
| Component       | Description                                                                          |
|-----------------|--------------------------------------------------------------------------------------|
| **Role**        | Agent's specialization (e.g., "Senior Researcher")                                   |
| **Goal**        | Primary objective (e.g., "Write technical blog posts")                               |
| **Backstory**   | Context shaping behavior (e.g., "Former MIT AI researcher")                          |
| **Tools**       | Capabilities the agent can use                                                       |
| **Verbose**     | Control logging level (True/False)                                                   |

#### **B. Task Structure**
```python
from crewai import Task

task = Task(
    description="Research latest AI trends",
    expected_output="Markdown report with 10 key trends",
    agent=researcher,
    tools=[search_tool]  # Specific tools for this task
)
```

#### **C. Crew Orchestration**
```python
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    process="sequential",  # Options: sequential, hierarchical
    memory=True,
    verbose=2
)
```

---

### **4. Official Best Practices**
1. **Tool Design**:
   - Keep tools atomic/single-purpose
   - Add clear docstrings for LLM understanding
   - Handle errors internally

2. **Memory Management**:
   - Use `memory=True` for complex workflows
   - Start with default embeddings
   - Monitor memory usage in logs

3. **Agent Specialization**:
   - Limit 3-5 tools per agent
   - Create specialist agents instead of generalists
   - Use distinct backstories for role separation

---

### **5. Troubleshooting from Docs**
**Common Errors**:
```python
# 1. Missing API keys
os.environ["OPENAI_API_KEY"] = "sk-your-key"  # Required
os.environ["SERPER_API_KEY"] = "serper-key"   # For search

# 2. Tool compatibility
@tool("New Tool")
def custom_tool(param: int):  # Type hints required
    ...

# 3. Memory initialization
from langchain_openai import OpenAIEmbeddings
crew = Crew(
    ...,
    memory=True,
    embedding_model=OpenAIEmbeddings()  # Must be configured
)
```
